## Webscraper for moneysavingexpert forum
    Created 1st July 2021
    Thanks for massive support to sydadder

In [2]:
# What to import
import requests
import pandas as pd
import json
import re
import requests as req
from bs4 import BeautifulSoup as soup
import datetime as dt
from datetime import timedelta
from dateutil.parser import parse


In [45]:
def mse_scrape(url, pages = 1, postdata = "off"):
    
    '''
    ### url: Define url in string format to pull request from
    ### pages: Define number of pages to search for threads on (default is first page only)
    ### postdata: Sequences through all threads in a page and pulls the body of the thread post and date first created. 
    ### Default is turn postdata to off as it can be slow to to run.
    '''
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

    
    ### Create dictionary for later creation of pandas DataFrame
    if postdata == "off":
        mse_questions = {"rowhref":[],
                     "rowtitle":[],
                     "viewcount":[],
                     "reply":[]}
        
        page = []
        for p in range(1, pages+1):
            page.append(f"{url}/p{p}")
        
        for link in page:
            urltoscrape = link
    
            reqs = req.get(urltoscrape, headers)
            pool = soup(reqs.content, 'html.parser')
            listofrows = pool.find_all("li", class_="ItemDiscussion")

            for listrows in listofrows:
                ### View Count
                for s in listrows.find_all('span', class_="ViewCount"):
                    for num in s.find_all('span', class_="Number"):
                        viewcount = num.get_text()
                ### Number of comments per post
                for r in listrows.find_all('span', class_="CommentCount"):
                    for comm in r.find_all('span', class_="Number"):
                        reply = comm.get_text()
                ### Get A Tag HREF AND TITLE
                for a in listrows.find_all('a', href=True):
                    if "forums.moneysavingexpert.com" in a['href']:
                        rowhref = a['href']
                        rowtitle = a.get_text()

                    ### Storing data in dictionary
                        mse_questions["rowhref"].append(rowhref)
                        mse_questions["rowtitle"].append(rowtitle)
                        mse_questions["viewcount"].append(viewcount)
                        mse_questions["reply"].append(reply)
    
        df_mse = pd.DataFrame(mse_questions)
        df_mse.to_csv("mse_questions.csv")
        print("CSV file saved in your local directory")
    
    
    ## Create dictionary for later creation of pandas DataFrame - with added body and date of irig
    if postdata == "on":
        mse_questions = {"rowhref":[],
                         "rowtitle":[],
                         "viewcount":[],
                         "reply":[],
                        "body":[],
                        "date_created":[]}
        
        page = []
        for p in range(1, pages+1):
            page.append(f"{url}/p{p}")
            
        for link in page:
            urltoscrape = link

            reqs = req.get(urltoscrape, headers)
            pool = soup(reqs.content, 'html.parser')
            listofrows = pool.find_all("li", class_="ItemDiscussion")

            for listrows in listofrows:
                ### View Count
                for s in listrows.find_all('span', class_="ViewCount"):
                    for num in s.find_all('span', class_="Number"):
                        viewcount = num.get_text()
                ### Number of comments per post
                for r in listrows.find_all('span', class_="CommentCount"):
                    for comm in r.find_all('span', class_="Number"):
                        reply = comm.get_text()
                ### Get A Tag HREF AND TITLE
                for a in listrows.find_all('a', href=True):
                    if "forums.moneysavingexpert.com" in a['href']:
                        rowhref = a['href']
                        rowtitle = a.get_text()

                        url = rowhref
                
                    reqs = req.get(url)
                    nextlevel = soup(reqs.content, 'html.parser')
                    body = nextlevel.find("div", class_="Message userContent")

                if body == None:
                    continue
                else:
                    result_body = body.get_text().strip()


                date_created = nextlevel.find("span", class_="MItem DateCreated").get_text().strip()

                if "Today" in date_created:
                    date_created = date_created.replace("Today", dt.datetime.now().strftime("%d %B"))
                    date_parsed = parse(date_created)

                else:
                    date_parsed = parse(date_created)

              ### Storing data in dictionary for later
                mse_questions["rowhref"].append(rowhref)
                mse_questions["rowtitle"].append(rowtitle)
                mse_questions["viewcount"].append(viewcount)
                mse_questions["reply"].append(reply)
                mse_questions["body"].append(result_body)
                mse_questions["date_created"].append(date_parsed)

        df_mse = pd.DataFrame(mse_questions)
        df_mse.to_csv("mse_questionsdetail.csv")
        print("CSV file saved in your local directory")
        
    return 